# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#create import file path
inputFile = "../Output/cities.csv"

#load dataframe
weatherCitiesDF = pd.read_csv(inputFile,keep_default_na=False)

#create output file

outputPath="../Output/"
# outputCsv= outputPath+"cities.csv"

weatherCitiesDF.loc[(weatherCitiesDF['City'] == 'Gobabis')].head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# List for holding cities
gmaps.configure(api_key=g_key)

#initiate the figure
fig = gmaps.figure()

#get data from dataframe
locations = weatherCitiesDF[["Lat", "Lng"]]
heat = weatherCitiesDF["Humidity"].astype(int)

#create heat layer
heat_layer = gmaps.heatmap_layer(locations
                                 ,weights=heat
                                 ,dissipating=False
                                 ,max_intensity=15
                                 ,point_radius=2)

#add layer
fig.add_layer(heat_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#create new dataframe based on the conditions in instructions
niceWeatherDF = weatherCitiesDF.loc[(weatherCitiesDF["Max Temp"].between(70,80))
                                    & (weatherCitiesDF["Wind Speed"] < 10.00)
                                    & (weatherCitiesDF["Cloudiness"] == 0) 
                                   ]
#remove NAs
niceWeatherDF.dropna(axis=0)
niceWeatherDF.describe() #sanity check

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,4.000000,4.000000,4.000000,4.000000,4.0,4.00000,4.000000e+00
mean,-3.434900,-7.013600,74.300000,65.500000,0.0,6.49500,1.619298e+09
std,24.909333,55.887833,3.446738,27.670682,0.0,2.89476,1.444054e+02
min,-19.843600,-71.337500,71.600000,27.000000,0.0,2.30000,1.619298e+09
25%,-18.190450,-44.801900,71.600000,54.750000,0.0,5.75750,1.619298e+09
50%,-13.618300,-0.558900,73.400000,73.500000,0.0,7.48000,1.619298e+09
75%,1.137250,37.229400,76.100000,84.250000,0.0,8.21750,1.619298e+09
max,33.340600,44.400900,78.800000,88.000000,0.0,8.72000,1.619299e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
nameL = []
latL = []
lngL = []
addyL = []
cityL = []
ctryL = []

for index, row in niceWeatherDF.iterrows():
    try:
        #cap lat and lng for city
        lat = row['Lat']
        lng = row['Lng']
    #     print(city) #sanity check
    #     print(f"Lat: {lat}") #sanity check
    #     print(f"Lng: {lng}") #sanity check

        #geocoordinates
        targetCoordinates = f"{lat} , {lng}"
        targetSearch = "hotel"
        targetRadius = 10000
        #targetType = "hotel"
        #print(targetCoordinates) #sanity check

        # set up a parameters dictionary
        params = {
            "location": targetCoordinates,
            "keyword": targetSearch,
            "radius": targetRadius,
            #"type": targetType,
            "key": g_key
            }

        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
        #get results
        response = requests.get(base_url, params=params).json()
        
        nameL.append(response['results'][0]['name'])
        latL.append(response['results'][0]['geometry']['location']['lat'])
        lngL.append(response['results'][0]['geometry']['location']['lng'])
        addyL.append(response['results'][0]['vicinity'])
        cityL.append(row['City'])
        ctryL.append(row['Country'])
        
    except KeyError:
        print(f"City or hotel not found")
        
    except IndexError:
        print(f"No hotel found")
      

No hotel found


In [11]:
hotelDict = {
    "Hotel Name" : nameL,
    "City" : cityL,
    "Hotel Lat" : latL,
    "Hotel Lng" : lngL,
    "Country" : ctryL,
    "Address" : addyL
} 

#convert dictionary to df
hotelDF = pd.DataFrame(hotelDict)

hotelDF

,Hotel Name,City,Hotel Lat,Hotel Lng,Country,Address
0,Apart Hotel Atenea,Ilo,-17.657635,-71.337780,PE,"Villa Marina Mz. 08 - Lote. 01, Ilo"
1,Babylon Rotana,Baghdad,33.291224,44.390489,IQ,"Karada Street, Baghdad"
2,Golden Peacock Resort Hotel,Beira,-19.827120,34.917726,MZ,Beira


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
infoBoxTemplate = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Address</dt><dd>{Address}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotelInfo = [infoBoxTemplate.format(**row) for index, row in hotelDF.iterrows()]
locations = hotelDF[["Hotel Lat", "Hotel Lng"]]

In [13]:
# Add marker layer ontop of heat map
fig 

markers = gmaps.marker_layer(locations, 
            info_box_content=hotelInfo)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))